## Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
# %pip install imbalanced-learn
# %pip install scikit-learn seaborn plotly
# %pip install xgboost
%pip install category_encoders

## Load from Kaggle

In [2]:
#### This is an optional code block                                  ####
#### please execute it once or none if you have the dataset uploaded ####

import kagglehub
import os
# Download latest version
#os.chdir('/content/sample_data')
path = kagglehub.dataset_download("xiaojiu1414/digix-global-ai-challenge")
# Print the current working directory
train_data=path+"/train/"
print("Current working directory: {0}".format(os.getcwd()))
# Change the current working directory
os.chdir(train_data)
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /Users/hochan.son/Project/MASDS/stats414
Current working directory: /Users/hochan.son/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/train


In [233]:
src= '/root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/train/train_data_ads.csv'
dst= '/content/sample_data/train_data_ads.csv'
#os.listdir('/root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/train')
#os.listdir('/content/sample_data')

import shutil
shutil.copyfile(src, dst)

# 2nd option
#shutil.copy(src, dst)


'/content/sample_data/train_data_ads.csv'

## Imports

In [103]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_score,recall_score,f1_score
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from IPython.display import display
from google.colab import drive
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.graph_objects import *
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [208]:
#check if the train_data_ads files exists, otherwise, connect google drive
# if os.path.isfile('train_data_ads.csv'):
#   path = os.getcwd()
# else:
#drive.mount('content/sample_data/')
path = '/content/sample_data/'
os.chdir(path)
pd.set_option('display.max_columns', None)
os.listdir()
##(memory useage: 1.6GB)

['anscombe.json',
 'README.md',
 'sampled_train_ads.csv',
 'ctgan_synthetic_blend.csv',
 'final_synthetic_ads_data.csv',
 'LLM_GPT2_synthetic_data.csv',
 'TVAE_synthetic_ads_data.csv',
 'california_housing_test.csv',
 'mnist_test.csv',
 'mnist_train_small.csv',
 'california_housing_train.csv']

## Read, Sample Data

In [217]:
sampled_df = pd.read_csv('sampled_train_ads.csv')
print(sampled_df['label'].value_counts())
sample_df.head()


label
0    302255
1    119136
Name: count, dtype: int64


,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,spread_app_id,hispace_app_tags,app_second_class,app_score,u_refreshTimes,u_feedLifeCycle,age_3,age_4,age_5,age_6,age_7,age_8,age_9,gender_3,gender_4,city_rank_3,city_rank_4,city_rank_5,series_group_3,series_group_4,series_group_5,series_group_6,series_group_7,series_group_8,net_type_3,net_type_4,net_type_5,net_type_6,net_type_7,creat_type_cd_3,creat_type_cd_4,creat_type_cd_5,creat_type_cd_6,creat_type_cd_7,creat_type_cd_8,creat_type_cd_9,creat_type_cd_10,inter_type_cd_3,inter_type_cd_4,inter_type_cd_5,app_score_6.0,app_score_10.0,residence_te,city_te,series_dev_te,emui_dev_te,device_name_te,device_size_te,adv_prim_id_te,slot_id_te,spread_app_id_te,hispace_app_tags_te,app_second_class_te,adv_id_te,day_of_week,pt_d_hour,pt_d_day,pt_d_month,pt_d_year,task_id_in_click1,adv_prim_id_in_click2,spread_app_id_in_click3,task_id_in_close1,adv_prim_id_in_close2,spread_app_id_in_close3,interest_98,interest_78,interest_112,interest_65,interest_168,interest_86,interest_0,interest_199,interest_219,interest_171,interest_17,interest_173,interest_195,interest_21,interest_216,interest_27,interest_10,interest_109,interest_39,interest_16,interest_108,interest_220,interest_169,interest_8,interest_44,interest_57,interest_25,interest_218,interest_63,interest_206,interest_50,interest_157,interest_96,interest_100,interest_143,interest_20,interest_140,interest_117,interest_119,interest_152,interest_51,interest_114,interest_41,interest_147,interest_74,interest_94,interest_116,interest_123,interest_49,interest_194,interest_71,interest_211,interest_62,interest_207,interest_131,interest_80,interest_128,interest_56,interest_15,interest_104,interest_153,interest_59,interest_142,interest_172,interest_79,interest_151,interest_191,interest_205,interest_30,interest_37
131191,0,194975.0,3.0,2.0,30.0,113.0,5.0,31.0,3.0,11.0,193.0,1332.0,4.0,10606.0,8.0,1288.0,5.0,59.0,190.0,47.0,14.0,10.0,9.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
253989,0,222074.0,5.0,2.0,16.0,391.0,2.0,16.0,5.0,35.0,191.0,2579.0,7.0,15986.0,8.0,1926.0,5.0,12.0,190.0,47.0,23.0,0.0,8.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
240274,0,260347.0,7.0,2.0,29.0,389.0,2.0,16.0,5.0,20.0,220.0,2431.0,7.0,12790.0,5.0,1206.0,3.0,38.0,213.0,18.0,23.0,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201820,0,214407.0,2.0,2.0,20.0,441.0,2.0,11.0,8.0,37.0,252.0,1205.0,7.0,21275.0,10.0,1203.0,4.0,16.0,162.0,47.0,14.0,10.0,5.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
76272,0,173019.0,5.0,2.0,34.0,278.0,5.0,30.0,3.0,13.0,272.0,1199.0,7.0,14240.0,5.0,2025.0,3.

In [215]:
def _generate_sampled_dataframe(random_seed=42,df=None, l0 = 1.0, l1= 1.0):
  if df is None:
    raise Exception("df is None")

  np.random.seed(random_seed)

  labels = df['label']
  unique_labels = labels.unique()

  # sample 10% from labels [0,1]
  # this sampling will be extracting only 10% of total popluations
  sampled_df = pd.DataFrame()
  for label in unique_labels:
    label_data = df[labels==label]
    if label == 0:
      label_sample = label_data.sample(
          n=int(len(label_data) * l0), # down sampled for the majority class
          random_state=42
      )
    else:
      label_sample = label_data.sample(
        n=int(len(label_data) * l1), # up sampled for the minority class
        random_state=42
      )
    sampled_df = pd.concat([sampled_df, label_sample])
  #print("count:{}".format(sampled_df['label'].value_counts(dropna=True)))
  return sampled_df

## Data Preprocessing, Cleaning

In [216]:
# TVAE synthetic data
synthetic_df = pd.read_csv('final_synthetic_ads_data.csv')
synthetic_df
#print("\nSynthetic: \n {}".format(synthetic_df['label'].value_counts()))

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,-0.501263,-0.125871,-0.023702,-1.577715,-0.514091,2.070909,0.837592,-1.032019,-1.733107,1.777527,1.348922,-0.224449,1.116992,0.525321,0.069770,1.619103,1.071830,0.083629,-0.376578,-0.924281,0.0,0.292593,0.946368,-0.191056,0.453023,-1.251427,-1.207700,-1.009470,-0.218744,-0.194499,-0.151434,0.053028,-1.304324,-0.610186,0.359821
1,-0.977034,-0.125871,0.692141,0.248349,2.201725,0.699966,0.475489,-0.245619,0.332696,-1.359107,-1.397352,-1.631986,-0.717257,0.525321,0.348700,-1.388898,0.041566,-1.374153,-0.376578,-0.226463,0.0,1.002802,-1.289998,-1.154316,0.453023,-1.251382,-1.207651,-1.009394,-0.218744,-0.194499,-0.151434,-1.455927,-1.304299,0.735449,0.359821
2,1.288486,-0.125871,0.189106,1.161380,-0.514091,0.383595,-1.655744,1.327181,0.849146,-0.836335,-0.220378,1.485643,0.331220,0.525321,1.176625,-0.783324,1.071830,-0.717082,-0.376578,-0.924281,0.0,0.606724,0.946368,-0.191056,0.453023,-1.251338,-1.207602,-1.009318,-0.218744,-0.194499,-0.151434,-1.440504,-1.304274,-0.610186,0.359821
3,0.453017,-0.125871,-0.237246,1.161380,-0.514091,-0.670976,-1.138454,0.540781,0.720034,-0.836335,0.825822,0.591134,1.220198,0.525321,-0.948062,1.084448,1.071830,1.061597,-0.376578,-0.924281,0.0,-0.745404,-0.827302,0.531389,0.453023,-1.251294,-1.207553,-1.009242,-0.218744,-0.194499,-0.151434,-1.470901,-1.304248,0.062632,0.359821
4,-0.541591,-0.125871,-1.434690,-1.577715,-0.514091,0.067224,-1.148800,1.327181,0.720034,-0.836335,-1.135802,0.512206,0.366404,0.525321,1.554941,-1.009409,1.071830,-0.087515,1.174058,-0.110160,0.0,-1.168798,0.792135,-0.913501,0.453023,-1.251249,-1.207504,-1.009166,-0.218744,-0.194499,-0.151434,0.051630,-1.304223,1.071858,0.359821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20207,1.525687,-0.125871,-1.436769,0.248349,2.201725,0.699966,0.475489,-0.245619,0.332696,-1.359107,-1.397352,-0.527003,-1.477228,0.525321,1.298110,-1.014438,0.041566,0.315897,-0.376578,1.052870,0.0,0.852565,0.946368,-0.191056,0.453023,-0.768880,-1.203821,-1.004828,-0.218744,-0.194499,-0.151434,1.040796,-1.010190,-0.610186,0.359821
20208,-0.394415,-0.125871,-1.686649,-1.121199,-0.514091,-0.776433,0.516872,-1.032019,-1.087544,0.209210,-0.089603,-0.408612,0.600963,0.525321,1.202096,-1.021228,1.071830,1.611703,1.174058,-1.040584,0.0,0.060409,-1.906926,2.698724,0.453023,-0.549214,-0.464577,-0.090982,-0.218744,-0.194499,-0.151434,0.068101,-0.863917,-0.610186,-3.422054
20209,1.381030,-0.125871,-0.317075,0.248349,2.201725,0.699966,0.475489,-0.245619,0.332696,-1.359107,-1.397352,-0.303376,0.504794,-2.862533,-1.119116,0.272154,-1.503829,-0.668183,-1.927215,1.576234,0.0,-0.308353,-1.444230,1.253834,-2.208576,-0.549170,-0.690269,-0.907044,-0.218744,-0.194499,-0.151434,0.536780,-0.863891,0.062632,0.359821
20210,0.819885,-0.125871,1.055535,0.704865,-0.514091,2.070909,-0.952230,-1.032019,0.849146,-0.836335,-0.089603,-0.303376,0.504794,-2.862533,1.211335,0.031484,0.041566,-0.014168,1.174058,1.052870,0.0,0.374540,0.715019,-0.672686,0.453023,-1.172443,-1.121912,-0.891824,-0.218744,-0.194499,-0.151434,1.021380,-0.863866,1.071858,0.359821


In [174]:
#PREPROCESSING / FEAT ENG
sample_df = _generate_sampled_dataframe(random_seed=10003,df=sampled_df, l0=0.8, l1=1.0)
print("\nSampled: \n {}".format(sample_df['label'].value_counts()))


Sampled: 
 label
0    241804
1    119136
Name: count, dtype: int64


In [224]:
#LLM GPT2 synthetic data
synthetic_df = pd.read_csv('LLM_GPT2_synthetic_data.csv')
synthetic_df.drop(columns=['Unnamed: 35'], inplace=True)
synthetic_df.head()
#print("\nSynthetic: \n {}".format(synthetic_df['label'].value_counts()))

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,6,1,192568,3,2,18,297,5,11,8,29,109,2032,7.0,14584,17683,7,1097,3,17,1,213,18,23,0,15603^30157^14584^14995^32000,2057^1611^1633^1690^1858,114^332^246^350,24107,1218,173,2022-06-06 1:06:00,112^65^173^152^109,6,17
1,6,1,111410,6,4,33,319,3,27,2,11,210,2117,4.0,17058,17683,8,1036,4,38,1,312,43,18,10,19574^18255^34164^30157^26873,1036^2066^1036^1863^1036,312^168^312^162,24107,1218,173,2022-06-06 11:08:00,219^168^168^16^27,7,17
2,6,1,163866,6,4,33,319,3,27,2,11,145,2397,7.0,14584,17683,8,1380,5,13,1,298,49,17,10,29073^15684^19448^14584^35131,1487^1112^1363^1036^1499,162^350^312^162^152,24107,1218,173,2022-06-06 11:50:00,78^65^168^98^112,9,17
3,6,1,178823,5,2,17,343,5,30,3,28,278,1555,7.0,26340,20934,8,1690,5,22,1,162,47,14,10,26355^23285^14584^16502^19502,1705^2066^1795^1342^1561,190^114^162^114^162,24107,1218,173,2022-06-04 7:01:00,21^8^199^199^98,9,17
4,1004,1,126516,7,2,46,354,2,31,3,20,346,2117,7.0,36132,20395,8,1637,4,17,1,372,20,13,10,26227^33386^29094^27009^23442,1236^1633^1637^1650^1114,168^257^372^162^301,24107,1218,173,2022-06-06 8:30:00,98^98^78^199^98,9,17


In [122]:
#CTGAN synthetic data
synthetic_df = pd.read_csv('ctgan_synthetic_blend.csv') #(approx. 2GB memory)
synthetic_sample_df = _generate_sampled_dataframe(random_seed=42, df=synthetic_df, l0=0, l1=1)

In [186]:
#synthetic_df = synthetic_df.drop(columns=['Unnamed: 0'])
print("\nSample: \n {}".format(sample_df['label'].value_counts()))
print("\nSynthetic: \n {}".format(synthetic_sample_df['label'].value_counts()))

combined_df = pd.concat([sample_df, synthetic_sample_df])
print(combined_df['label'].value_counts())


Sample: 
 label
0    241804
1    119136
Name: count, dtype: int64

Synthetic: 
 label
1    148746
Name: count, dtype: int64
label
1    267882
0    241804
Name: count, dtype: int64


In [187]:
combined_df.drop(columns=['Unnamed: 0'], inplace=True)
#combined_df.drop(combined_df.columns[0], axis=1, inplace=True)

combined_df.head()


,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle,age_3,age_4,age_5,age_6,age_7,age_8,age_9,gender_3,gender_4,city_rank_3,city_rank_4,city_rank_5,series_group_3,series_group_4,series_group_5,series_group_6,series_group_7,series_group_8,net_type_3,net_type_4,net_type_5,net_type_6,net_type_7,creat_type_cd_3,creat_type_cd_4,creat_type_cd_5,creat_type_cd_6,creat_type_cd_7,creat_type_cd_8,creat_type_cd_9,creat_type_cd_10,inter_type_cd_3,inter_type_cd_4,inter_type_cd_5,app_score_6.0,app_score_10.0,residence_te,city_te,series_dev_te,emui_dev_te,device_name_te,device_size_te,adv_prim_id_te,slot_id_te,spread_app_id_te,hispace_app_tags_te,app_second_class_te,adv_id_te
131191,49484.0,0,194975.0,3.0,2.0,30.0,113.0,5.0,31.0,3.0,11.0,193.0,1332.0,4.0,15219.0,10606.0,8.0,1288.0,5.0,59.0,1.0,190.0,47.0,14.0,10.0,31706^17020^26531^20851^23844,1036^2039^1535^1407,312^190^168,31706,1036,312,2022-06-07 07:05:00,219^65^98^168^205,9.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253989,363733.0,0,222074.0,5.0,2.0,16.0,391.0,2.0,16.0,5.0,35.0,191.0,2579.0,7.0,29697.0,15986.0,8.0,1926.0,5.0,12.0,1.0,190.0,47.0,23.0,0.0,30157^25313^32409^18921^22871,2066^1112^1169^1291^1433,114^190^321^168^372,24107,1218,173,2022-06-04 06:03:00,98^98^112^98^26,8.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240274,1019423.0,0,260347.0,7.0,2.0,29.0,389.0,2.0,16.0,5.0,20.0,220.0,2431.0,7.0,14180.0,12790.0,5.0,1206.0,3.0,38.0,1.0,213.0,18.0,23.0,0.0,24066^13533^15146,1645^1391^1220,280,31941^23523^24159^35359^15661,1036^1037^2025^1391^1044,312^327^298^152^132,2022-06-07 03:40:00,86^16^220^147^219,3.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201820,508144.0,0,214407.0,2.0,2.0,20.0,441.0,2.0,11.0,8.0,37.0,252.0,1205.0,7.0,10655.0,21275.0,10.0,1203.0,4.0,16.0,1.0,162.0,47.0,14.0,10.0,25802^17173^29887^23523,1037^1220^1856,327^280^162,35196,1795,173,2022-06-03 07:58:00,168^157^37^112^207,5.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76272,367138.0,0,173019.0,5.0,2.0,34.0,278.0,5.0,30.0,3.0,13.0,272.0,1199.0,7.0,24159.0,14240.0,5.0,2025.0,3.0,59.0,1.0,213.0,18.0,23.0,0.0,17034^26861^27404^30416^11711,2076^1637^1863^1049^1795,123^152^190,24107,1218,173,2022-06-03 08:45:00,30^220^168^211^128,6.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
# def preprocess_data(sample_df):
#     if 'pt_d' in sample_df.columns:
#         # make pt_d useful
#         # Assuming missing values are represented as empty strings
#         sample_df['pt_d'] = sample_df['pt_d'].replace('', np.nan)  # Replace empty strings with NaN
#         sample_df['pt_d'] = pd.to_datetime(sample_df['pt_d'], format='%Y%m%d%H%M', errors='coerce')

#         sample_df['day_of_week'] = sample_df['pt_d'].dt.dayofweek

#         # Extract numerical features from 'pt_d'
#         sample_df['pt_d_hour'] = sample_df['pt_d'].dt.hour
#         sample_df['pt_d_day'] = sample_df['pt_d'].dt.day
#         sample_df['pt_d_month'] = sample_df['pt_d'].dt.month
#         sample_df['pt_d_year'] = sample_df['pt_d'].dt.year

#         # Now drop the original 'pt_d' column
#         sample_df.drop(columns=['pt_d'], inplace=True)
#     else:
#         print("Warning: 'pt_d' column not found in the DataFrame.")

#     # convert carat lists to python lists
#     ad_lst_cols = ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003','ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003', 'u_newsCatInterestsST']
#     for col in ad_lst_cols:
#         #sample_df[col] = sample_df[col].apply(lambda x: list(map(int, x.split("^"))))
#         sample_df[col] = sample_df[col].apply(lambda x: list(map(int, str(x).split("^"))) if isinstance(x, str) else [])

#     # create boolean fields for ad ID matching
#     def check_id_match(id_value, id_list):
#         return int(id_value in id_list)

#     sample_df["task_id_in_click1"] = sample_df.apply(lambda row: check_id_match(row["task_id"], row["ad_click_list_v001"]), axis=1)
#     sample_df["adv_prim_id_in_click2"] = sample_df.apply(lambda row: check_id_match(row["adv_prim_id"], row["ad_click_list_v002"]), axis=1)
#     sample_df["spread_app_id_in_click3"] = sample_df.apply(lambda row: check_id_match(row["spread_app_id"], row["ad_click_list_v003"]), axis=1)
#     sample_df["task_id_in_close1"] = sample_df.apply(lambda row: check_id_match(row["task_id"], row["ad_close_list_v001"]), axis=1)
#     sample_df["adv_prim_id_in_close2"] = sample_df.apply(lambda row: check_id_match(row["adv_prim_id"], row["ad_close_list_v002"]), axis=1)
#     sample_df["spread_app_id_in_close3"] = sample_df.apply(lambda row: check_id_match(row["spread_app_id"], row["ad_close_list_v003"]), axis=1)

#     # news cat interest top-k
#     all_interests = [item for sublist in sample_df["u_newsCatInterestsST"] if isinstance(sublist, list) for item in sublist]
#     news_cat_thresh = 70  # user set
#     top_k = [x[0] for x in Counter(all_interests).most_common(news_cat_thresh)]

#     # Create binary features
#     for interest in top_k:
#         sample_df[f"interest_{interest}"] = sample_df["u_newsCatInterestsST"].apply(lambda x: 1 if isinstance(x, list) and interest in x else 0)

#     # return both sample_df and top_k
#     return sample_df, top_k

In [189]:
def preprocess_data(sample_df):
    if 'pt_d' in sample_df.columns:
        # make pt_d useful
        # Assuming missing values are represented as empty strings
        sample_df['pt_d'] = sample_df['pt_d'].replace('', np.nan)  # Replace empty strings with NaN
        sample_df['pt_d'] = pd.to_datetime(sample_df['pt_d'], format='%Y%m%d%H%M', errors='coerce')

        sample_df['day_of_week'] = sample_df['pt_d'].dt.dayofweek

        # Extract numerical features from 'pt_d'
        sample_df['pt_d_hour'] = sample_df['pt_d'].dt.hour
        sample_df['pt_d_day'] = sample_df['pt_d'].dt.day
        sample_df['pt_d_month'] = sample_df['pt_d'].dt.month
        sample_df['pt_d_year'] = sample_df['pt_d'].dt.year

        # Now drop the original 'pt_d' column
        sample_df.drop(columns=['pt_d'], inplace=True)
    else:
        print("Warning: 'pt_d' column not found in the DataFrame.")

    # convert carat lists to python lists
    ad_lst_cols = ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003', 'u_newsCatInterestsST']
    for col in ad_lst_cols:
        # Check if the column exists before processing it
        if col in sample_df.columns:
            # Check if the column contains any non-numeric values before applying int()
            # If it does, handle those values appropriately (e.g., skip them, replace them with a default value)
            sample_df[col] = sample_df[col].apply(lambda x: list(map(int, str(x).split("^"))) if isinstance(x, str) and x.replace('^', '').isdigit() else [])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame.")

    # create boolean fields for ad ID matching
    def check_id_match(id_value, id_list):
        return int(id_value in id_list)

    sample_df["task_id_in_click1"] = sample_df.apply(lambda row: check_id_match(row["task_id"], row["ad_click_list_v001"]), axis=1)
    sample_df["adv_prim_id_in_click2"] = sample_df.apply(lambda row: check_id_match(row["adv_prim_id"], row["ad_click_list_v002"]), axis=1)
    sample_df["spread_app_id_in_click3"] = sample_df.apply(lambda row: check_id_match(row["spread_app_id"], row["ad_click_list_v003"]), axis=1)
    sample_df["task_id_in_close1"] = sample_df.apply(lambda row: check_id_match(row["task_id"], row["ad_close_list_v001"]), axis=1)
    sample_df["adv_prim_id_in_close2"] = sample_df.apply(lambda row: check_id_match(row["adv_prim_id"], row["ad_close_list_v002"]), axis=1)
    sample_df["spread_app_id_in_close3"] = sample_df.apply(lambda row: check_id_match(row["spread_app_id"], row["ad_close_list_v003"]), axis=1)

    # news cat interest top-k
    all_interests = [item for sublist in sample_df["u_newsCatInterestsST"] if isinstance(sublist, list) for item in sublist]
    news_cat_thresh = 70  # user set
    top_k = [x[0] for x in Counter(all_interests).most_common(news_cat_thresh)]

    # Create binary features
    for interest in top_k:
        sample_df[f"interest_{interest}"] = sample_df["u_newsCatInterestsST"].apply(lambda x: 1 if isinstance(x, list) and interest in x else 0)

    # return both sample_df and top_k
    return sample_df, top_k

In [190]:
sample_df, top_k = preprocess_data(combined_df)
##(Memory Usage: 5.1GB)

In [191]:
#bars and hists
colors = ['#B4DCEA', '#df6164', '#5f77ac', '#ffc7cc',  '#50d1e0']
layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

In [192]:
columns_to_plot = ['age', 'gender', 'u_refreshTimes']

for col in columns_to_plot:
    # Calculate mean CTR for each group
    mean_ctr = sample_df.groupby(col)['label'].mean().reset_index()
    mean_ctr.rename(columns={'label': 'mean_ctr'}, inplace=True)

    # Create the bar plot
    fig = px.bar(mean_ctr, x=col, y='mean_ctr', title=f'Mean CTR vs {col}')
    fig.update_traces(marker_color='#B4DCEA')
    fig.update_layout(layout, width=800, height=400, showlegend=False)
    fig.add_hline(y=0.015, line_dash="dash", line_color="red", line_width=1,
                  annotation_text="global mean",
              annotation_position="top right")

    fig.show()

    #histogram
    fig = px.histogram(sample_df, x=col, nbins=50, title=f'Distribution of {col}')
    fig.update_layout(
        xaxis_title=col,
        yaxis_title='Frequency',
        showlegend=True
    )
    fig.update_traces(marker_color='#5f77ac')
    fig.update_layout(layout, width=800, height=400, showlegend=False)
    fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [193]:
# One-Hot Encoding for Low Cardinality Categorical Features
one_hot_categorical_columns = [
    "age", "gender", "city_rank", "series_group", "net_type",
    "creat_type_cd", "inter_type_cd", "app_score"
]

# Initialize OneHotEncoder (from sklearn)
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)

# Drop unnecessary columns and separate features and target variable
sample_df.drop(columns=['log_id', 'task_id', 'site_id', 'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
                         'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003', 'u_newsCatInterestsST'],
                inplace=True, errors='ignore')

# Separate features (X) and target variable (y)
y = sample_df['label']
X = sample_df.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Fit and transform on training data
X_train_onehot = onehot_encoder.fit_transform(X_train[one_hot_categorical_columns])

# Convert One-Hot Encoded features to DataFrame with correct column names
onehot_feature_names = onehot_encoder.get_feature_names_out(one_hot_categorical_columns)
X_train_processed = pd.DataFrame(X_train_onehot, index=X_train.index, columns=onehot_feature_names)

# 2. K-Fold Target Encoding (with smoothing)
target_encoding_categorical_columns = [
    "residence", "city", "series_dev", "emui_dev", "device_name",
    "device_size", "adv_prim_id", "slot_id", "spread_app_id",
    "hispace_app_tags", "app_second_class", "adv_id"
]

alpha_config = {
    "device_name": 25, "slot_id": 25, "spread_app_id": 25,
    "city": 15, "adv_prim_id": 15, "hispace_app_tags": 15,
    "residence": 8, "series_dev": 8, "emui_dev": 8,
    "device_size": 3, "app_second_class": 3, "adv_id": 15
}

global_mean = y_train.mean()  # Global mean of the target variable
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store the target encoding mappings for each column
target_encoding_mappings = {}

# Iterate over each column for target encoding
for col in target_encoding_categorical_columns:
    # Create a new column for target encoding in the training set
    X_train_processed[f'{col}_te'] = np.nan

    # K-Fold Encoding for the training set
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr = y_train.iloc[train_idx]

        # Compute the smoothed mean for each category
        group_sum = y_tr.groupby(X_tr[col]).sum()
        group_count = y_tr.groupby(X_tr[col]).count()
        alpha = alpha_config[col]  # for the current column
        encoding = (group_sum + global_mean * alpha) / (group_count + alpha)

        # Store the encoding for the column in the dictionary for later use
        target_encoding_mappings[col] = encoding

        # Reset index of X_val to avoid duplicate index issues
        X_val = X_val.reset_index(drop=True)

        # Instead of using X_val.index, use the actual indices from the original X_train DataFrame
        X_train_processed.loc[X_train.index[val_idx], f'{col}_te'] = X_val[col].map(encoding)


    # Fill missing values in the training set with the global mean
    X_train_processed[f'{col}_te'].fillna(global_mean, inplace=True)



# Now you have the target encoding mappings and one-hot encoder saved.
# These can be used later to apply the same encoding to the test set.

# Merge the target variable back into the processed training data
train_final = X_train_processed.copy()
train_final['label'] = y_train

# Now `train_final` contains the one-hot and target encoded features, and you can use `onehot_encoder` and `target_encoding_mappings` to encode the test data later.
## (Memory usage: 6.6GB)

## Correlation Plots

In [194]:

# Compute the correlation matrix for numerical features
corr_matrix = train_final.corr(numeric_only=True)
label_corr = corr_matrix['label'].drop('label').sort_values(ascending=False)

print("\nFeature correlations with label (top 20 in absolute value):")
print(label_corr.abs().sort_values(ascending=False).head(20))

# Select the top 30 features based on absolute correlation with the label
top_features = label_corr.abs().sort_values(ascending=False).head(30).index.tolist()

# Create a sub-correlation matrix for the selected features
sub_corr_matrix = corr_matrix.loc[top_features, top_features]

mask = np.triu(np.ones(sub_corr_matrix.shape), k=1)  # Upper triangle mask (excluding diagonal)

# Apply the mask
corr_matrix_masked = sub_corr_matrix.copy()
corr_matrix_masked = np.where(mask, np.nan, sub_corr_matrix)

## (Memory usage: 6.8GB)


Feature correlations with label (top 20 in absolute value):
city_rank_nan          0.610027
age_nan                0.610027
creat_type_cd_nan      0.610027
series_group_nan       0.610027
inter_type_cd_nan      0.610027
app_score_nan          0.610027
gender_nan             0.610027
net_type_nan           0.610027
app_score_10.0         0.457221
net_type_7.0           0.390721
creat_type_cd_10.0     0.366292
inter_type_cd_4.0      0.283657
residence_te           0.249097
emui_dev_te            0.245676
city_te                0.245618
series_dev_te          0.240919
app_second_class_te    0.239323
inter_type_cd_5.0      0.237230
device_name_te         0.236951
hispace_app_tags_te    0.230967
Name: label, dtype: float64


In [195]:

# Use Plotly Express to create the heatmap
fig = px.imshow(
    corr_matrix_masked,
    # text_auto=True,                  # Automatically annotate the cells with the correlation values
    color_continuous_scale='RdPu',
    zmin=-1,                         # Minimum value for the color scale
    zmax=1,                          # Maximum value for the color scale
    labels=dict(x="Features", y="Features", color="Correlation"),
    title="Top 30 Features Correlation with Label"
)

# Rotate x-axis labels for better readability
fig.update_xaxes(ticktext=top_features, tickvals=list(range(len(top_features))))
fig.update_yaxes(ticktext=top_features, tickvals=list(range(len(top_features))))
fig.update_layout(layout, xaxis=dict(tickangle=45), width=800, height=600)

# Display the interactive heatmap
fig.show()

In [196]:
#corr mat showing only > 0.6

threshold = 0.6
corr_matrix_filtered = sub_corr_matrix.where(abs(sub_corr_matrix) >= threshold)

#Remove NaN values and display
corr_matrix_filtered = corr_matrix_filtered.dropna(how='all')
corr_matrix_filtered = corr_matrix_filtered.dropna(axis=1, how='all')

# print(corr_matrix_filtered)
mask = np.triu(np.ones(corr_matrix_filtered.shape), k=1)  # Upper triangle mask (excluding diagonal)

# Apply the mask
corr_matrix_filtered_masked = corr_matrix_filtered.copy()
corr_matrix_filtered_masked = np.where(mask, np.nan, corr_matrix_filtered)


# Use Plotly Express to create the heatmap
fig = px.imshow(
    corr_matrix_filtered_masked,
    # text_auto=True,                  # Automatically annotate the cells with the correlation values
    color_continuous_scale='RdPu',
    zmin=-1,                         # Minimum value for the color scale
    zmax=1,                          # Maximum value for the color scale
    labels=dict(x="Features", y="Features", color="Correlation"),
    title="Top 30 Features Correlation with Label"
)

# Rotate x-axis labels for better readability
fig.update_xaxes(ticktext=top_features, tickvals=list(range(len(top_features))))
fig.update_yaxes(ticktext=top_features, tickvals=list(range(len(top_features))))
fig.update_layout(layout, xaxis=dict(tickangle=45), width=800, height=600)

# Display the interactive heatmap
fig.show()

In [197]:
# Get absolute correlation values and stack into a long format
corr_pairs = sub_corr_matrix.abs().stack().reset_index()
corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# Keep only unique pairs (avoid self-correlations and duplicates)
corr_pairs = corr_pairs[corr_pairs['Feature_1'] != corr_pairs['Feature_2']]
corr_pairs = corr_pairs[corr_pairs['Correlation'] >= 0.6]  # Apply threshold
corr_pairs = corr_pairs.drop_duplicates(subset=['Correlation'])

# Sort by correlation value (descending)
corr_pairs = corr_pairs.sort_values(by='Correlation', ascending=False)

# Display the resulting DataFrame
corr_pairs


,Feature_1,Feature_2,Correlation
1,city_rank_nan,age_nan,1.000000
374,residence_te,city_te,0.986057
468,series_dev_te,device_name_te,0.982348
373,residence_te,emui_dev_te,0.982224
405,emui_dev_te,series_dev_te,0.972417
...,...,...,...
479,series_dev_te,adv_id_te,0.667151
419,emui_dev_te,adv_id_te,0.665239
389,residence_te,adv_id_te,0.664314
569,device_name_te,adv_id_te,0.663901


In [198]:
label_corr = label_corr.reset_index()
label_corr = label_corr.rename(columns={'label': 'label_corr'})

label_corr['label_corr'] = label_corr['label_corr'].abs()

# Merge to add label_corr_1 (correlation between Feature_1 and label)
corr_pairs = corr_pairs.merge(
    label_corr, left_on='Feature_1', right_on='index', how='left')
corr_pairs.drop(columns=['index'], inplace=True)
corr_pairs.rename(columns={'label_corr':'label_corr_1'}, inplace=True)
corr_pairs.head()

# Merge to add label_corr_1 (correlation between Feature_1 and label)
corr_pairs = corr_pairs.merge(
    label_corr, left_on='Feature_2', right_on='index', how='left')
corr_pairs.drop(columns=['index'], inplace=True)
corr_pairs.rename(columns={'label_corr':'label_corr_2'}, inplace=True)
corr_pairs.head()


,Feature_1,Feature_2,Correlation,label_corr_1,label_corr_2
0,city_rank_nan,age_nan,1.000000,0.610027,0.610027
1,residence_te,city_te,0.986057,0.249097,0.245618
2,series_dev_te,device_name_te,0.982348,0.240919,0.236951
3,residence_te,emui_dev_te,0.982224,0.249097,0.245676
4,emui_dev_te,series_dev_te,0.972417,0.245676,0.240919


In [199]:
corr_pairs['Dropped_Feature'] = corr_pairs.apply(
    lambda row: row['Feature_1'] if row['label_corr_1'] < row['label_corr_2'] else row['Feature_2'],
    axis=1
)

corr_drop = corr_pairs.Dropped_Feature.tolist()


In [201]:
# #clean it up
train_final = train_final.drop(columns=corr_drop)
gc.collect()
# ##(memory usage: approx. 6.7 GB)

KeyError: "['age_nan', 'city_te', 'device_name_te', 'emui_dev_te', 'series_dev_te', 'city_te', 'hispace_app_tags_te', 'series_dev_te', 'device_name_te', 'spread_app_id_te', 'app_second_class_te', 'series_dev_te', 'device_name_te', 'app_second_class_te', 'app_second_class_te', 'device_name_te', 'app_second_class_te', 'spread_app_id_te', 'device_size_te', 'hispace_app_tags_te', 'device_size_te', 'device_size_te', 'device_name_te', 'device_size_te', 'hispace_app_tags_te', 'hispace_app_tags_te', 'device_size_te', 'hispace_app_tags_te', 'adv_prim_id_te', 'hispace_app_tags_te', 'device_size_te', 'spread_app_id_te', 'spread_app_id_te', 'adv_id_te', 'spread_app_id_te', 'device_size_te', 'spread_app_id_te', 'adv_prim_id_te', 'spread_app_id_te', 'spread_app_id_te', 'adv_id_te', 'adv_prim_id_te', 'slot_id_te', 'adv_prim_id_te', 'adv_prim_id_te', 'adv_prim_id_te', 'adv_prim_id_te', 'adv_prim_id_te', 'adv_prim_id_te', 'app_score_10.0', 'adv_id_te', 'slot_id_te', 'slot_id_te', 'slot_id_te', 'slot_id

## 	Train Test Split

In [202]:
sample_labels = train_final['label']
train_final.drop(columns='label', inplace=True)

In [203]:
X_train, X_test, y_train, y_test = train_test_split(
    train_final, sample_labels, test_size=0.2, random_state=42
)
# ##(memory usage: approx. 6.6 GB)

## Model fit and Performance Metrics

In [204]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, precision_score,mean_absolute_error,recall_score

def _fit_models(models, X_train, y_train, X_test, y_test):

    results = {}

    for name, model in models.items():
        if name != 'ensemble':  # Skip 'ensemble' in this loop
            print("\n now training model: >>{}<<".format(model)) # Change here: access the model object directly
            model.fit(X_train, y_train)

            _y_pred_proba = model.predict_proba(X_test)[:, 1]
            _y_pred = model.predict(X_test)
            _fpr, _tpr, _thresholds = roc_curve(y_test, _y_pred_proba)
            _roc_auc = auc(_fpr, _tpr)
            _roc_auc_score = roc_auc_score(y_test, _y_pred_proba)
            _confusion_matrix = confusion_matrix(y_test, _y_pred)
            _classification_report = classification_report(y_test, _y_pred, zero_division=0)
            _f1_score = f1_score(y_test, _y_pred, zero_division=0)
            _rmse = np.sqrt(np.mean((y_test - _y_pred) ** 2))
            _Rsquare = model.score(X_test, y_test)
            _accuracy = accuracy_score(y_test, _y_pred)
            _precision = precision_score(y_test, _y_pred, zero_division=0)
            _recall = recall_score(y_test, _y_pred, zero_division=0)
            _mae = mean_absolute_error(y_test, _y_pred)



            if hasattr(model, 'feature_importances_'):
                _feature_importance = model.feature_importances_
            # For Logistic Regression, use coefficients instead
            else:
                _feature_importance = model.coef_[0]

            results[name] = {
                'auc_roc': _roc_auc_score,
                'fpr': _fpr,
                'tpr': _tpr,
                'thresholds': _thresholds,
                'roc_auc': _roc_auc,
                'f1_score': _f1_score,
                'feature_importance': _feature_importance,
                'model': model,
                'confusion_matrix': _confusion_matrix,
                'classification_report': _classification_report,
                'rmse': _rmse,
                'Rsquare': _Rsquare,
                'accuracy': _accuracy,
                'recall': _recall,
                'precision': _precision,
                'MAE': _mae
            }
        elif name == 'ensemble':
            print("\n now training model: >>models['ensemble']<<") # Change here: print 'Ensemble'

            # Create list of (name, estimator) tuples for StackingClassifier
            estimators = []
            for model_name in models['ensemble'][:-1]:  # All but the last model
                estimators.append((model_name, results[model_name]['model']))

            # Get the final estimator (last model in the ensemble list)
            final_estimator = results[models['ensemble'][-1]]['model']

            # Create and fit StackingClassifier
            _ensemble_model = StackingClassifier(
                estimators=estimators,
                final_estimator=final_estimator,
                cv=5
            )
            _ensemble_model.fit(X_train, y_train)
            _y_pred_ensemble = _ensemble_model.predict(X_test)
            _ensemble_fpr, _ensemble_tpr, _ensemble_thresholds = roc_curve(y_test, _y_pred_ensemble)
            _ensemble_accuracy = accuracy_score(y_test, _y_pred_ensemble)
            # Store the ensemble model results
            results['ensemble'] = {
                'model': _ensemble_model,
                'accuracy': accuracy_score(y_test, _y_pred_ensemble),
                'classification_report': classification_report(y_test, _y_pred_ensemble),
                'confusion_matrix': confusion_matrix(y_test, _y_pred_ensemble),
                'auc_roc': roc_auc_score(y_test, _ensemble_model.predict_proba(X_test)[:, 1]),
                'fpr': _ensemble_fpr,
                'tpr': _ensemble_tpr,
                'roc_auc': auc(_ensemble_fpr, _ensemble_tpr),
                'thresholds': _ensemble_thresholds,
                'f1_score': f1_score(y_test, _y_pred_ensemble),
                'feature_importance': None,
                'rmse': np.sqrt(np.mean((y_test - _y_pred_ensemble) ** 2)),
                'Rsquare': _ensemble_model.score(X_test, y_test),
                'accuracy': _ensemble_accuracy,
                'percision': precision_score(y_test, _y_pred_ensemble),
                'recall': recall_score(y_test, _y_pred_ensemble),
                'MAE': mean_absolute_error(y_test, _y_pred_ensemble)
            }
    return results

In [205]:
## model fitting

fit_results = {}
# model1: logistics regression only
# model1 = {
#     'logistic': LogisticRegression(random_state=42)
# }
# fit_results['model1'] = _fit_models(model1, X_train, y_train, X_test, y_test)

# model2: XGBoost only
model2 = {
    'xgboost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
}
fit_results['model2'] = _fit_models(model2, X_train, y_train, X_test, y_test)

# # model3: 1.Lasso -> 2.logistic regression -> 3.ensemble: {Stacking}
# model3 = {
#     'logistic_lasso': LogisticRegression(penalty='l1', solver='liblinear', random_state=42),
#     'logistic': LogisticRegression(random_state=42),
#     'ensemble': ['logistic_lasso','logistic']
# }
# fit_results['model3'] = _fit_models(model3, X_train, y_train, X_test, y_test)
##(memory usage: approx. 7.4 GB)


 now training model: >>XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)<<


## Metrics visualization

In [206]:
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

for x, results in fit_results.items():
    for name, result in results.items():
        print("\nModel: {}-{}".format(x, name))
        print(f"AUC-ROC: {result['auc_roc']}")
        print(f"F1 Score: {result['f1_score']}")
        # Check if feature_importance is not None before proceeding
        if result['feature_importance'] is not None:
            print(f"Feature Importance: {result['feature_importance']}")
            print("confusion:{}".format(result['confusion_matrix']))
            print("classification_report:\n{}".format(result['classification_report']))
            print("rmse:{}".format(result['rmse']))
            print("Rsquare:{}".format(result['Rsquare']))
            print("\n")

            # Confusion Matrix with Plotly
            cm = confusion_matrix(y_test, result['model'].predict(X_test))  # Access using name as key
            fig_cm = px.imshow(cm,
                               labels=dict(x="Predicted", y="Actual"),
                               x=['1', '0'],
                               y=['1', '0'],
                               title=f"Confusion Matrix - {x}-{name}",
                               text_auto=True,
                               color_continuous_scale='Teal')
            fig_cm.update_layout(layout, height=600, width=600)
            fig_cm.show()

            feature_importance = results[name]['feature_importance']
            feature_importance = np.abs(feature_importance)
            sorted_idx = np.argsort(feature_importance)[::-1]
            feature_importance = feature_importance[sorted_idx][:30]
            feature_names = X_train.columns  # Assuming X_train has feature names
            feature_names = feature_names[sorted_idx][:30]

            # Feature Importance Bar Plot with Plotly
            fig_fi = px.bar(x=feature_names, y=feature_importance,
                            labels={'x': 'Features', 'y': 'Importance'},
                            title=f"Top 30 Feature Importance - {x}-{name}", )
            fig_fi.update_traces(marker_color="#5f77ac")
            fig_fi.update_layout(layout, xaxis=dict(tickangle=45, tickfont=dict(size=10)),
                                 width=700, height=600)
            fig_fi.show()

        # ROC Curve with Plotly
        fig_roc = go.Figure(data=[go.Scatter(x=result['fpr'], y=result['tpr'],
                                            mode='lines',
                                            name='ROC Curve',
                                            line=dict(color='darkorange', width=2)),
                                  go.Scatter(x=[0, 1], y=[0, 1],
                                            mode='lines',
                                            name='Random Classifier',
                                            line=dict(color='navy', width=2, dash='dash'))])
        fig_roc.update_layout(layout, title=f"Receiver Operating Characteristic - {x}-{name}",
                              xaxis_title='False Positive Rate',
                              yaxis_title='True Positive Rate',
                              showlegend=True, width=700, height=600)
        fig_roc.show()


Model: model2-xgboost
AUC-ROC: 0.8937306055815306
F1 Score: 0.7772923479398299
Feature Importance: [6.8663247e-04 2.9281678e-04 2.5951241e-03 4.2417948e-04 1.8855766e-04
 3.5894426e-04 4.6437641e-04 1.9017507e-04 4.2390596e-04 8.1356537e-01
 1.7662642e-04 1.7149074e-04 2.5103230e-04 0.0000000e+00 6.5187929e-04
 2.3104835e-04 1.4644218e-04 5.6504557e-04 3.2696463e-04 1.9145985e-04
 0.0000000e+00 3.8507651e-04 4.8485791e-04 8.5434898e-05 5.5369874e-04
 0.0000000e+00 2.0206084e-04 1.1704628e-04 2.4990164e-04 3.5659927e-03
 5.2467623e-04 4.0159412e-04 8.0817394e-05 3.5056237e-02 0.0000000e+00
 3.9197929e-04 4.4744271e-03 2.0828647e-04 0.0000000e+00 7.0588234e-05
 0.0000000e+00 1.3124530e-01]
confusion:[[34812  3904]
 [13122 29712]]
classification_report:
              precision    recall  f1-score   support

           0       0.73      0.90      0.80     38716
           1       0.88      0.69      0.78     42834

    accuracy                           0.79     81550
   macro avg       0

In [151]:
cm =confusion_matrix(y_test,fit_results['model3']['ensemble']['model'].predict(X_test))

fig_cm = px.imshow(cm,
                    labels=dict(x="Predicted", y="Actual"),
                    x=['1', '0'],
                    y=['1', '0'],
                    title=f"Confusion Matrix - {x}-{name}",
                    text_auto=True,
                    color_continuous_scale='Teal')
fig_cm.update_layout(layout, height=600, width=600)
fig_cm.show()

KeyError: 'model3'

In [207]:
import pandas as pd

# Assuming 'fit_results' contains the results from your model training

# Create an empty list to store the results
results_data = []

# Iterate through the fit_results dictionary
for model_name, model_results in fit_results.items():
    for model_variation, result in model_results.items():
        results_data.append([
            model_name + '-' + model_variation,  # Model name
            result['MAE'],                # MAE
            result['rmse'],                   # RMSE
            result['Rsquare'],                 # R-squared
            result['accuracy'],                # Accuracy
            result['precision'],              # precision
            result['recall'],                  # recall
            result['f1_score']                  # F1 Score

        ])

# Create a pandas DataFrame from the results data
results_df = pd.DataFrame(results_data, columns=['Model','MAE', 'RMSE', 'R-squared','accuracy','precision','recall','F1 Score'])

# Display the DataFrame
display(results_df)

,Model,MAE,RMSE,R-squared,accuracy,precision,recall,F1 Score
0,model2-xgboost,0.20878,0.456924,0.79122,0.79122,0.883865,0.693655,0.777292


## Cross-Validation

In [ ]:
X_sample, y_sample = X_train, y_train
# Define logistic_model and xgb_model before the loop
logistic_model = LogisticRegression(random_state=42)
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

xgb_f1_scores = []
logistic_f1_scores = []

for train_idx, val_idx in kfold.split(X_sample, y_sample):
    X_train_fold, X_val_fold = X_sample.iloc[train_idx], X_sample.iloc[val_idx]
    y_train_fold, y_val_fold = y_sample.iloc[train_idx], y_sample.iloc[val_idx]

    # Train and Evaluate Logistic Regression
    logistic_model.fit(X_train_fold, y_train_fold)
    y_pred_logistic = logistic_model.predict(X_val_fold)
    f1_logistic = f1_score(y_val_fold, y_pred_logistic, average='weighted')
    logistic_f1_scores.append(f1_logistic)

    # Train and Evaluate XGBoost
    xgb_model.fit(X_train_fold, y_train_fold)
    y_pred_xgb = xgb_model.predict(X_val_fold)
    f1_xgb = f1_score(y_val_fold, y_pred_xgb, average='weighted')
    xgb_f1_scores.append(f1_xgb)

#Compute Mean F1 Scores
logistic_mean_f1 = np.mean(logistic_f1_scores)
xgb_mean_f1 = np.mean(xgb_f1_scores)

#  Results
print(f'Logistic Regression Weighted F1 Score: {logistic_mean_f1}')
print(f'XGBoost Weighted F1 Score: {xgb_mean_f1}')


Logistic Regression Weighted F1 Score: 0.6760980263647114
XGBoost Weighted F1 Score: 0.6776692258663998


## Test Data

In [ ]:
path = kagglehub.dataset_download("xiaojiu1414/digix-global-ai-challenge")
# Print the current working directory
test_data=path+"/test/"
print("Current working directory: {0}".format(os.getcwd()))
# Change the current working directory
os.chdir(test_data)
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/train
Current working directory: /root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/test


In [ ]:
test_ads = pd.read_csv('test_data_ads.csv') #(approx. 2GB memory)
test_ads.head()

,log_id,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,242295,100002,2,3,13,225,2,31,3,21,210,2177,4,14584,17683,5,1236,3,22,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101201,86^15^86^112^191,0,15
1,242300,100002,2,3,13,225,2,31,3,21,210,2419,6,14584,17683,5,1236,3,13,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101154,86^15^86^112^191,0,15
2,242299,100002,2,3,13,225,2,31,3,21,210,2177,4,35178,10779,8,1060,4,58,1,344,20,13,10.0,21791^33265,2020^1236,352,24107,1218,173,202206101228,86^15^86^112^191,0,15
3,242296,100002,2,3,13,225,2,31,3,21,210,1031,6,14584,17683,5,1236,3,17,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101144,86^15^86^112^191,0,15
4,242293,100002,2,3,13,225,2,31,3,21,210,2177,4,19474,21876,8,2051,5,31,1,186,15,22,10.0,21791^33265,2020^1236,352,24107,1218,173,202206101145,86^15^86^112^191,0,15


In [ ]:
test_preprocessed = preprocess_data(test_ads, top_k_interests=top_k)

TypeError: preprocess_data() got an unexpected keyword argument 'top_k_interests'

In [ ]:
X_test = test_preprocessed.copy()

# 1. One-Hot Encoding for Low Cardinality Categorical Features
# Apply the one-hot encoding transformation to the test data (using the same encoder from training data)
X_test_onehot = onehot_encoder.transform(X_test[one_hot_categorical_columns])

# Convert One-Hot Encoded features to DataFrame with correct column names
onehot_feature_names = onehot_encoder.get_feature_names_out(one_hot_categorical_columns)
X_test_processed = pd.DataFrame(X_test_onehot, index=X_test.index, columns=onehot_feature_names)

# 2. Apply Target Encoding (using target_encoding_mappings from training data)
# Apply the target encoding to the test set using the precomputed target encoding mappings

for col in target_encoding_categorical_columns:
    # Apply target encoding from training data to test set
    X_test_processed[f'{col}_te'] = X_test[col].map(target_encoding_mappings.get(col, {}))

    # Fill missing values in the test set with the global mean (from training data)
    X_test_processed[f'{col}_te'].fillna(global_mean, inplace=True)

# Now X_test_processed has both one-hot encoded features and target encoded features
# You can proceed with training a model or evaluation with X_test_processed

In [ ]:
X_test_final = X_test_processed.drop(columns=corr_drop)

In [ ]:
# Predict on the test set using the trained XGBoost model
y_test_pred_xgb = xgb_model.predict(X_test_final)

# Create a DataFrame with the predictions
# Assuming the test set doesn't have the label column
submission_df = pd.DataFrame({
    'id': X_test_final.index,  # If your test set has an 'id' column, use it, otherwise use the index
    'prediction': y_test_pred_xgb
})

In [ ]:
submission_df.prediction.value_counts()

In [ ]:
submission_df.to_csv('xgb_predictions.csv', index=False)

from google.colab import files

# Download the output CSV file to local machine
files.download('xgb_predictions.csv')

In [ ]:
#ENCODING
# # Define categorical columns for different encoding strategies
# one_hot_categorical_columns = [
#     "age",
#     "gender",
#     "city_rank",
#     "series_group",
#     "net_type",
#     "creat_type_cd",
#     "inter_type_cd",
#     "app_score"
# ]

# target_encoding_categorical_columns = [
#     "residence",
#     "city",
#     "series_dev",
#     "emui_dev",
#     "device_name",
#     "device_size",
#     "adv_prim_id",
#     "slot_id",
#     "spread_app_id",
#     "hispace_app_tags",
#     "app_second_class",
#     'adv_id'
# ]

# sample_df.drop(columns=['log_id', 'task_id', 'site_id', 'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
#                  'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003', 'u_newsCatInterestsST'],
#         inplace=True, errors='ignore')

# # Separate features and target variable
# y = sample_df['label']
# X = sample_df.drop(columns=['label'])

# # Split the dataset into train and test sets (ensure stratification for balanced class distribution)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, stratify=y, random_state=42
# )

# #One-Hot Encoding for Low Cardinality Categorical Features
# onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
# X_train_onehot = onehot_encoder.fit_transform(X_train[one_hot_categorical_columns])

# # Use X_test instead of X_train when transforming the test set
# X_test_onehot = onehot_encoder.transform(X_test[one_hot_categorical_columns])

# # Convert One-Hot Encoded features to DataFrame with correct column names
# onehot_feature_names = onehot_encoder.get_feature_names_out(one_hot_categorical_columns)
# X_train_processed = pd.DataFrame(X_train_onehot, index=X_train.index, columns=onehot_feature_names)
# X_test_processed = pd.DataFrame(X_test_onehot, index=X_test.index, columns=onehot_feature_names)

# # 2. K-Fold Target Encoding (with smoothing)
# alpha_config = {
#     "device_name": 25,
#     "slot_id": 25,
#     "spread_app_id": 25,
#     "city": 15,
#     "adv_prim_id": 15,
#     "hispace_app_tags": 15,
#     "residence": 8,
#     "series_dev": 8,
#     "emui_dev": 8,
#     "device_size": 3,
#     "app_second_class": 3,
#     "adv_id": 15
# }

# global_mean = y_train.mean()
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# target_encoding_mappings = {}

# for col in target_encoding_categorical_columns:
#     # Create a new column for target encoding in the training set
#     X_train_processed[f'{col}_te'] = np.nan

#     # K-Fold Encoding for the training set
#     for train_idx, val_idx in kf.split(X_train):
#         X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#         y_tr = y_train.iloc[train_idx]

#         # Compute the smoothed mean for each category
#         group_sum = y_tr.groupby(X_tr[col]).sum()
#         group_count = y_tr.groupby(X_tr[col]).count()
#         alpha = alpha_config[col]  # for the current column
#         encoding = (group_sum + global_mean * alpha) / (group_count + alpha)

#         # Store the encoding for the column in the dictionary for later use
#         target_encoding_mappings[col] = encoding

#         # Map the computed encoding values to the validation fold
#         X_train_processed.loc[X_val.index, f'{col}_te'] = X_val[col].map(encoding)

#     # Fill missing values in the training set with the global mean
#     X_train_processed[f'{col}_te'].fillna(global_mean, inplace=True)

#     # Encode the test set using the entire training data
#     group_sum_full = y_train.groupby(X_train[col]).sum()
#     group_count_full = y_train.groupby(X_train[col]).count()
#     encoding_full = (group_sum_full + global_mean * alpha) / (group_count_full + alpha)
#     X_test_processed[f'{col}_te'] = X_test[col].map(encoding_full)
#     X_test_processed[f'{col}_te'].fillna(global_mean, inplace=True)

# # Merge the target variable back into the processed training data.
# train_final = X_train_processed.copy()
# train_final['label'] = y_train